In [40]:
from LamIndexer import LamIndexer
from os import listdir
from PIL import Image
from calendar import isleap
import numpy as np
import pandas as pd

# Mitä tarvitaan:
# - tiedostot
# - tiedot mikä indeksi on mikä lamId


In [57]:
selected_year = 2016
days_in_selected_year = 366 if isleap(2016) else 365

day_range = range(1, days_in_selected_year + 1)
image_file_path = f"YOUR_PATH\\data\\processed_image_files\\{selected_year}"
day_path = lambda day: f"{image_file_path}\\{day}"
feature_names = ["CarCount", "CarSpeedMean", "CarSpeedMax", "CarSpeedMin"]
lam_indexer = LamIndexer()

In [62]:
def create_dataframe(img_list, day_index, year, feature_names):
  lam_ids_list = []
  directions_list = []
  data_list = []
  for i in range(len(img_list)):
    img = img_list[i]
    width, height = img.size
    lam_ids = []
    directions = []
    data = []
    img_bytes = img.tobytes()
    for i in range(width):
      is_empty = img_bytes[(height-2) * width + i] == 255
      if not is_empty:
          col = img_bytes[i:(height-2)*width:width]
          lam_id = lam_indexer.get_lam_id_for_index(i)
          direction = (i % 2) + 1
          lam_ids.append(lam_id)
          directions.append(direction)
          data.append(np.array(bytearray(col), dtype="uint"))
    lam_ids_list.append(lam_ids)      
    directions_list.append(directions)      
    data_list.append(data) 
  # sanity check
  if len(set([len(ids) for ids in lam_ids_list])) != 1:
    print("ERROR!!!")
  
  df_data = {"lamId": lam_ids_list[0], "dayIndex": day_index, "year": year, "direction": directions_list[0]}
  for i in range(len(feature_names)):
    feat = feature_names[i]
    df_data[feat] = data_list[i]

  result_df = pd.DataFrame(df_data)
  return result_df


  

In [63]:
def handle_day(day_index, feature_names, year):
  day_p = day_path(day_index)
  files_for_day = listdir(day_p)
  day_data = []
  for file in files_for_day:
    file_split = file.split("_")
    if file_split[0] in feature_names:
      img = Image.open(f"{day_p}\\{file}")
      day_data.append(img)
  if len(day_data) == len(feature_names):
    return create_dataframe(day_data, day_index, year, feature_names)
  else:
    print(f"Couldn't find all features for day {day_index}")
  
  

In [73]:
full_year_data = []
for day in day_range:
  result = handle_day(day, feature_names, selected_year)
  full_year_data.append(result)

In [74]:
full_df = pd.concat(full_year_data)

In [77]:
pickle_save_path = f"YOUR_PATH\\data\\pickels_v2\\aggregated_car_data_{selected_year}.pkl"
full_df.to_pickle(pickle_save_path)